In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
import sys

In [2]:
from tqdm import tqdm

In [3]:
sys.path.append("C:/Users/User/Desktop/hopfield-izhikevich/hopfield/")
from hopfield import Hopfield
sys.path.pop();

In [4]:
def classify(image, dataset, score)-> int:
    max_i = 0
    for i in range(len(dataset)):
        sample = dataset[i]
        if score(image, sample) > score(image, dataset[max_i]):
            max_i = i
    return max_i    

In [5]:
def dice_score(prediction, data) -> float:
    tp, tn, fp, fn = 0, 0, 0, 0
    for p, d in zip(prediction.flat, data.flat):
        if d>0 and p>0:
            tp += 1
        # if d<0 and p<0:
        #     tn += 1
        if d>0 and p<0:
            fn += 1
        if d<0 and p>0:
            fp += 1
  
    return 2*tp/(2*tp + fp + fn)

In [6]:
def iou_score(prediction, data) -> float:
    tp, tn, fp, fn = 0, 0, 0, 0
    for p, d in zip(prediction.flat, data.flat):
        if d>0 and p>0:
            tp += 1
        # if d<0 and p<0:
        #     tn += 1
        if d>0 and p<0:
            fn += 1
        if d<0 and p>0:
            fp += 1
  
    return tp/(tp + fp + fn)

In [7]:
dataset_iter = tfds.as_numpy(tfds.load('mnist', split='train', shuffle_files=True, as_supervised=True))
dataset = np.fromiter(dataset_iter, dtype=tuple)
dataset_iter_test = tfds.as_numpy(tfds.load('mnist', split='train', shuffle_files=True, as_supervised=True))
test = np.fromiter(dataset_iter_test, dtype=tuple)

In [8]:
digits = [[] for i in range(10)]
index = 0
for d in dataset:
    digits[d[1]].append(index)
    index += 1

In [9]:
memory_images = []
scale = 0.6
threshold = 100
shape = ()
for digit in range(10):
    index = random.choice(digits[ digit ])
    image = tf.image.central_crop(dataset[index][0], scale)
    image = tf.where(image>threshold, 1, 0)
    memory_images.append(image.numpy().flatten())

    
shape = tuple(image.shape)[:2]
memory_images = np.array(memory_images, dtype=np.float32)*2-1


In [10]:
test_count = 1000
scale = 0.6
q = 0
method='hebb'

result_dice = [[] for i in range(10)]
result_iou = [[] for i in range(10)]

model = Hopfield()
model.train(memory_images, method=method)

for index in tqdm(range(test_count)):
    image, label = test[index]
    image = tf.image.central_crop(image, scale)
    image = tf.where(image>threshold, 1, 0)
    image = image.numpy().flatten()
    image = np.array(image, np.float32)*2-1
    
    output = model.run(image, q=q)
    
    predicted_label_dice = classify(output['output'], memory_images, dice_score)
    predicted_label_iou = classify(output['output'], memory_images, iou_score)
    
    result_dice[label].append( predicted_label_dice == label )
    result_iou[label].append( predicted_label_iou == label )

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [02:24<00:00,  6.92it/s]


In [11]:
i = 0
print("Result with Dice(F1) score classifier")
result_dice = np.array(result_dice, dtype=object)
for stat in result_dice:
    print(f"{i}: {int(sum(stat)/len(stat)*100)}%, {len(stat)} tests")
    i+=1
print(f" total: {sum([sum(i) for i in result_dice])/test_count*100}%")

Result with Dice(F1) score classifier
0: 15%, 84 tests
1: 0%, 109 tests
2: 0%, 100 tests
3: 0%, 96 tests
4: 0%, 113 tests
5: 98%, 85 tests
6: 0%, 92 tests
7: 0%, 94 tests
8: 0%, 112 tests
9: 0%, 115 tests
 total: 9.700000000000001%
